# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('data/kc_house_data.gl/')

[INFO] 1450837061 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/luozhiyi/bin/anaconda2/lib/python2.7/site-packages/certifi/cacert.pem
1450837061 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to jessherlock@sjtu.edu.cn and will expire on October 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-10140 - Server binary: /home/luozhiyi/bin/anaconda2/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1450837061.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

** Following Explanation is Interesting**
* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2.0,0


In [6]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.503457     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.573725     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [33]:
import numpy as np
coef=model_all.get("coefficients")
#coef.print_rows(num_rows=len(coef))
num = (coef['value']).nnz()
nnz_coef = coef[coef['value'].apply(lambda x : not np.isclose(x,0))]
nnz_coef.print_rows(num_rows=num)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
+------------------+-------+---------------+
[6 rows x 3 columns]



Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

constant, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [34]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [37]:
sales.column_names()

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'sqft_living_sqrt',
 'sqft_lot_sqrt',
 'bedrooms_square',
 'floors_square']

In [51]:
import math

my_output = 'price'
output = validation[my_output]
best_l1penalty, best_RSS = .0, .0
flag = True

for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target=my_output, features=all_features, validation_set=None, l2_penalty=0, l1_penalty=l1_penalty, verbose=None)
    predictions = model.predict(validation)
    RSS = ((predictions - output)**2).sum()
    if flag:
        best_l1penalty = l1_penalty
        best_RSS = RSS
        flag = False
    else:
        if l1_penalty < best_l1penalty:
            best_l1penalty = l1_penalty
            best_RSS = RSS
    print "l1_penalty = 10 **",math.log10(l1_penalty),"RSS: ",RSS

print "best_l1penalty:",best_l1penalty, "best_RSS:", best_RSS


l1_penalty = 10 ** 1.0 RSS:  6.25766285142e+14
l1_penalty = 10 ** 1.5 RSS:  6.25766285362e+14
l1_penalty = 10 ** 2.0 RSS:  6.25766286058e+14
l1_penalty = 10 ** 2.5 RSS:  6.25766288257e+14
l1_penalty = 10 ** 3.0 RSS:  6.25766295212e+14
l1_penalty = 10 ** 3.5 RSS:  6.25766317206e+14
l1_penalty = 10 ** 4.0 RSS:  6.25766386761e+14
l1_penalty = 10 ** 4.5 RSS:  6.25766606749e+14
l1_penalty = 10 ** 5.0 RSS:  6.25767302792e+14
l1_penalty = 10 ** 5.5 RSS:  6.25769507644e+14
l1_penalty = 10 ** 6.0 RSS:  6.25776517727e+14
l1_penalty = 10 ** 6.5 RSS:  6.25799062845e+14
l1_penalty = 10 ** 7.0 RSS:  6.25883719085e+14
best_l1penalty: 10.0 best_RSS: 6.25766285142e+14


In [52]:
best_model = graphlab.linear_regression.create(training, target=my_output, features=all_features, validation_set=None, l2_penalty=0, l1_penalty=10.0, verbose=None)
test_RSS = ( (testing[my_output]  - best_model.predict(testing)) ** 2).sum()
print test_RSS

1.56983602382e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?

** 10.0 **

2. What is the RSS on TEST data of the model with the best `l1_penalty`?

**  1.56983602382e+14 **


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [59]:
print len(all_features), best_model.get("coefficients")['value'].nnz()

17 18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [61]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [62]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [66]:
my_output = 'price'
output = validation[my_output]
best_l1penalty, best_RSS = .0, .0
flag = True

l1_penalty_max, l1_penalty_min = .0, .0
small = True
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target=my_output, features=all_features, validation_set=None, l2_penalty=0, l1_penalty=l1_penalty, verbose=None)
    nnzs = model['coefficients']['value'].nnz()
    
    if nnzs >= max_nonzeros: 
        l1_penalty_max = l1_penalty
    if nnzs <= max_nonzeros and small:
        l1_penalty_min = l1_penalty
        small = False
        
    predictions = model.predict(validation)
    RSS = ((predictions - output)**2).sum()
    if flag:
        best_l1penalty = l1_penalty
        best_RSS = RSS
        flag = False
    else:
        if l1_penalty < best_l1penalty:
            best_l1penalty = l1_penalty
            best_RSS = RSS
    print "l1_penalty = 10 **",math.log10(l1_penalty),"RSS: ",RSS, "non-zero parameters: ", nnzs

print "best_l1penalty:",best_l1penalty, "best_RSS:", best_RSS


l1_penalty = 10 ** 8.0 RSS:  6.27492659875e+14 non-zero parameters:  18
l1_penalty = 10 ** 8.10526315789 RSS:  6.28210516771e+14 non-zero parameters:  18
l1_penalty = 10 ** 8.21052631579 RSS:  6.29176689541e+14 non-zero parameters:  18
l1_penalty = 10 ** 8.31578947368 RSS:  6.30650082719e+14 non-zero parameters:  18
l1_penalty = 10 ** 8.42105263158 RSS:  6.32940229287e+14 non-zero parameters:  17
l1_penalty = 10 ** 8.52631578947 RSS:  6.3626814023e+14 non-zero parameters:  17
l1_penalty = 10 ** 8.63157894737 RSS:  6.41261198311e+14 non-zero parameters:  17
l1_penalty = 10 ** 8.73684210526 RSS:  6.48983455376e+14 non-zero parameters:  17
l1_penalty = 10 ** 8.84210526316 RSS:  6.60962217696e+14 non-zero parameters:  17
l1_penalty = 10 ** 8.94736842105 RSS:  6.77261520728e+14 non-zero parameters:  16
l1_penalty = 10 ** 9.05263157895 RSS:  7.01046815867e+14 non-zero parameters:  15
l1_penalty = 10 ** 9.15789473684 RSS:  7.37850622829e+14 non-zero parameters:  15
l1_penalty = 10 ** 9.263157

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [67]:
print " l1_penalty_min:", l1_penalty_min, " l1_penalty_max:", l1_penalty_max 

 l1_penalty_min: 3792690190.73  l1_penalty_max: 2976351441.63


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

** 3792690190.73, 2976351441.63 **

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [68]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [69]:
flag = True
my_output = 'price'
output = validation[my_output]
best_l1penalty, best_RSS = .0, .0

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target=my_output, features=all_features, validation_set=None, l2_penalty=0, l1_penalty=l1_penalty, verbose=None)
    predictions = model.predict(validation)
    nnzs = model['coefficients']['value'].nnz()
    RSS = ((predictions - output)**2).sum()
    
    if nnzs==max_nonzeros:
        if  flag:
            best_l1penalty = l1_penalty
            best_RSS = RSS
            flag = False
        else:
            if l1_penalty < best_l1penalty:
                best_l1penalty = l1_penalty
                best_RSS = RSS
    print "l1_penalty = 10 **",math.log10(l1_penalty),"RSS: ",RSS, "non-zero parameters: ", nnzs

print "best_l1penalty:",best_l1penalty, "best_RSS:", best_RSS

l1_penalty = 10 ** 9.57894736842 RSS:  1.08186759232e+15 non-zero parameters:  6
l1_penalty = 10 ** 9.57399941753 RSS:  1.07763277558e+15 non-zero parameters:  6
l1_penalty = 10 ** 9.56899444397 RSS:  1.07350454959e+15 non-zero parameters:  6
l1_penalty = 10 ** 9.56393111807 RSS:  1.06946433543e+15 non-zero parameters:  6
l1_penalty = 10 ** 9.55880806313 RSS:  1.0657076895e+15 non-zero parameters:  6
l1_penalty = 10 ** 9.55362385314 RSS:  1.06079953176e+15 non-zero parameters:  7
l1_penalty = 10 ** 9.54837701038 RSS:  1.05599273534e+15 non-zero parameters:  7
l1_penalty = 10 ** 9.54306600295 RSS:  1.05114762561e+15 non-zero parameters:  7
l1_penalty = 10 ** 9.53768924203 RSS:  1.04693748875e+15 non-zero parameters:  7
l1_penalty = 10 ** 9.53224507905 RSS:  1.04323723787e+15 non-zero parameters:  8
l1_penalty = 10 ** 9.52673180266 RSS:  1.03855473594e+15 non-zero parameters:  8
l1_penalty = 10 ** 9.52114763549 RSS:  1.03461690923e+15 non-zero parameters:  8
l1_penalty = 10 ** 9.51549073

In [70]:
print max_nonzeros

7


** l1_penalty = 10 ** 9.53768924203 RSS:  1.04693748875e+15 non-zero parameters:  7 **

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?    best_l1penalty: 3448968612.19702
2. What features in this model have non-zero coefficients?

** constant, bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above **


In [71]:
model = graphlab.linear_regression.create(training, target=my_output, features=all_features, validation_set=None, l2_penalty=0, l1_penalty=best_l1penalty, verbose=None)
coef = model.get('coefficients')
num = (coef['value']).nnz()
nnz_coef = coef[coef['value'].apply(lambda x : not np.isclose(x,0))]
nnz_coef.print_rows(num_rows=num)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
+------------------+-------+---------------+
[7 rows x 3 columns]

